# Nafisur Rahman
nafisur21@gmail.com<br>
https://www.linkedin.com/in/nafisur-rahman

## Document Similarity

In [1]:
import numpy as np
import pandas as pd
import sklearn
import nltk
import matplotlib.pyplot as plt
from nltk import pos_tag
from nltk.tokenize import word_tokenize

In [2]:
tags=pos_tag(word_tokenize('Fish are nvqjp friends.'))
tags

[('Fish', 'NN'),
 ('are', 'VBP'),
 ('nvqjp', 'JJ'),
 ('friends', 'NNS'),
 ('.', '.')]

In [5]:
nltk.help.upenn_tagset('N')

NN: noun, common, singular or mass
    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhumanity
    machinist ...
NNP: noun, proper, singular
    Motown Venneboerger Czestochwa Ranzer Conchita Trumplane Christos
    Oceanside Escobar Kreisler Sawyer Cougar Yvette Ervin ODI Darryl CTCA
    Shannon A.K.C. Meltex Liverpool ...
NNPS: noun, proper, plural
    Americans Americas Amharas Amityvilles Amusements Anarcho-Syndicalists
    Andalusians Andes Andruses Angels Animals Anthony Antilles Antiques
    Apache Apaches Apocrypha ...
NNS: noun, common, plural
    undergraduates scotches bric-a-brac products bodyguards facets coasts
    divestitures storehouses designs clubs fragrances averages
    subjectivists apprehensions muses factory-jobs ...


In [3]:
from nltk.corpus import wordnet as wn

In [4]:
wn.synsets('fish')

[Synset('fish.n.01'),
 Synset('fish.n.02'),
 Synset('pisces.n.02'),
 Synset('pisces.n.01'),
 Synset('fish.v.01'),
 Synset('fish.v.02')]

In [6]:
for s in wn.synsets('fish'):
    print(s,s.definition())

Synset('fish.n.01') any of various mostly cold-blooded aquatic vertebrates usually having scales and breathing through gills
Synset('fish.n.02') the flesh of fish used as food
Synset('pisces.n.02') (astrology) a person who is born while the sun is in Pisces
Synset('pisces.n.01') the twelfth sign of the zodiac; the sun is in this sign from about February 19 to March 20
Synset('fish.v.01') seek indirectly
Synset('fish.v.02') catch or try to catch fish or shellfish


In [7]:
for s in wn.synsets('fish',pos='v'):
    print(s,s.definition())

Synset('fish.v.01') seek indirectly
Synset('fish.v.02') catch or try to catch fish or shellfish


In [8]:
def convert_tag(tag):
    """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
    
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None

In [9]:
for tag in tags:
    #print(tag)
    #print(convert_tag(tag[1]))
    #print(wn.synsets(tag[0],pos=convert_tag(tag[1])))
    try:
        print(wn.synsets(tag[0],pos=convert_tag(tag[1]))[0])
    except:
        None

Synset('fish.n.01')
Synset('be.v.01')
Synset('friend.n.01')


In [10]:
synset_list=[]
for tag in tags:
    try:
        ss=wn.synsets(tag[0],pos=convert_tag(tag[1]))[0]
        synset_list.append(ss)
    except:
        None
    
    
print(synset_list)

[Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]


In [11]:
from bs4 import BeautifulSoup
import re
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
stemmer=SnowballStemmer('english')
from nltk.stem import WordNetLemmatizer
lemma=WordNetLemmatizer()
from nltk.corpus import stopwords
from string import punctuation
Cstopwords=set(stopwords.words('english')+list(punctuation))
def doc_to_synsets(doc):
    text=str(doc)
    text=re.sub('[^a-zA-Z]',' ',text)
    text=word_tokenize(text)
    #text=[w for w in text if w not in Cstopwords]
    #text=[stemmer.stem(w) for w in text if w not in Cstopwords]
    #text=[lemma.lemmatize(w) for w in text if w not in Cstopwords]
    tags=pos_tag(text)
    synset_list=[]
    for tag in tags:
        try:
            ss=wn.synsets(tag[0],pos=convert_tag(tag[1]))[0]
            synset_list.append(ss)
        except:
            None
    return synset_list

In [12]:
doc_to_synsets('Fish are nvqjp friends.')

[Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]

In [13]:
synsets1 = doc_to_synsets('I like cats')
synsets2 = doc_to_synsets('I like dogs')
print(synsets1)
print(synsets2)

[Synset('iodine.n.01'), Synset('wish.v.02'), Synset('cat.n.01')]
[Synset('iodine.n.01'), Synset('wish.v.02'), Synset('dog.n.01')]


In [14]:
def similarity_score(s1, s2):
    """
    Calculate the normalized similarity score of s1 onto s2

    For each synset in s1, finds the synset in s2 with the largest similarity value.
    Sum of all of the largest similarity values and normalize this value by dividing it by the
    number of largest similarity values found.

    Args:
        s1, s2: list of synsets from doc_to_synsets

    Returns:
        normalized similarity score of s1 onto s2

    Example:
        synsets1 = doc_to_synsets('I like cats')
        synsets2 = doc_to_synsets('I like dogs')
        similarity_score(synsets1, synsets2)
        Out: 0.73333333333333339
    """
    
    list_of_largest_similarities = []
    # Your Code Here
    for syn1 in s1:
        
        largest_sim = 0
        
        for syn2 in s2:
            
            #print(syn1.path_similarity(syn2))
            
            if syn1.path_similarity(syn2):
            
                if syn1.path_similarity(syn2) > largest_sim:
                
                    largest_sim = syn1.path_similarity(syn2)
                
        list_of_largest_similarities.append(largest_sim)
    
    return sum(list_of_largest_similarities) / len(list_of_largest_similarities)


In [15]:
print(similarity_score(doc_to_synsets('i like mangoes'), synsets2))

0.6969696969696969


In [16]:
print(similarity_score(synsets1, synsets2))

0.7333333333333334


In [17]:
def document_path_similarity(doc1, doc2):
    """Finds the symmetrical similarity between doc1 and doc2"""

    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)

    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2

In [18]:
def test_document_path_similarity():
    doc1 = 'This is a function to test document_path_similarity.'
    doc2 = 'Use this function to see if your code in doc_to_synsets \
    and similarity_score is correct!'
    return document_path_similarity(doc1, doc2)

paraphrases is a DataFrame which contains the following columns: Quality, D1, and D2.

Quality is an indicator variable which indicates if the two documents D1 and D2 are paraphrases of one another (1 for paraphrase, 0 for not paraphrase).

In [19]:
# Use this dataframe for questions most_similar_docs and label_accuracy
paraphrases = pd.read_csv('paraphrases.csv')
paraphrases.head()

,Quality,D1,D2
0,1,"Ms Stewart, the chief executive, was not expec...","Ms Stewart, 61, its chief executive officer an..."
1,1,After more than two years' detention under the...,After more than two years in detention by the ...
2,1,"""It still remains to be seen whether the reven...","""It remains to be seen whether the revenue rec..."
3,0,"And it's going to be a wild ride,"" said Allan ...","Now the rest is just mechanical,"" said Allan H..."
4,1,The cards are issued by Mexico's consulates to...,The card is issued by Mexico's consulates to i...


In [20]:
paraphrases['D1'][0]

'Ms Stewart, the chief executive, was not expected to attend.'

In [21]:
paraphrases['D2'][0]

'Ms Stewart, 61, its chief executive officer and chairwoman, did not attend.\n'

In [22]:
pd.options.display.max_colwidth = 100

In [23]:
paraphrases

,Quality,D1,D2
0,1,"Ms Stewart, the chief executive, was not expected to attend.","Ms Stewart, 61, its chief executive officer and chairwoman, did not attend.\n"
1,1,"After more than two years' detention under the State Security Bureau, the four were found guilty...","After more than two years in detention by the State Security Bureau, the four were found guilty ..."
2,1,"""It still remains to be seen whether the revenue recovery will be short- or long-lived,"" Sprayre...","""It remains to be seen whether the revenue recovery will be short- or long-lived,"" said James Sp..."
3,0,"And it's going to be a wild ride,"" said Allan Hoffenblum, a Republican consultant.","Now the rest is just mechanical,"" said Allan Hoffenblum, a Republican consultant.\n"
4,1,The cards are issued by Mexico's consulates to its citizens living abroad and show the date of b...,The card is issued by Mexico's consulates to its citizens living abroad and shows the date of bi...
5,1,Their difference was over whether the court should pay attention to legal opinions of other worl...,Their difference was over whether the court should take into account the legal opinions of other...
6,1,"The only announced Republican to replace Davis is Rep. Darrell Issa of Vista, who has spent $1.7...","So far the only declared major party candidate is Rep. Darrell Issa, a Republican who has spent ..."
7,1,"Druce will face murder charges, Conte said.",Conte said Druce will be charged with murder.\n
8,0,"""It's a major victory for Maine, and it's a major victory for other states.",The Maine program could be a model for other states.\n
9,1,Microsoft said Friday that it is halting development of future Macintosh versions of its Interne...,Microsoft will stop developing versions of its Internet Explorer browser software for Macintosh ...


In [24]:
def most_similar_docs():
    
    
    s = 0.0
    for i in range(len(paraphrases)):
        similarity = document_path_similarity(paraphrases['D1'][i], paraphrases['D2'][i])
        if s < similarity:
            s = similarity
            result = ((paraphrases['D1'][i], paraphrases['D2'][i], similarity))
    return result

In [25]:
def label_accuracy():
    from sklearn.metrics import accuracy_score

    
    label=[]
    for i in range(len(paraphrases)):
        similarity = document_path_similarity(paraphrases['D1'][i], paraphrases['D2'][i])
        if similarity > 0.75:
            label.append(1)
        else:
            label.append(0)
        
    return accuracy_score(paraphrases['Quality'],label)

In [26]:
most_similar_docs()

("The cards are issued by Mexico's consulates to its citizens living abroad and show the date of birth, a current photograph and the address of the card holder.",
 "The card is issued by Mexico's consulates to its citizens living abroad and shows the date of birth, a current photograph and the address of the cardholder.\n",
 0.9301413255360624)

In [27]:
label_accuracy()

0.75